In [1]:
from __future__ import print_function, division

from keras.preprocessing import image
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, Lambda, Multiply, Add
from keras.layers import BatchNormalization, Activation, ZeroPadding2D, Concatenate
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D, MaxPooling2D, AveragePooling2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
#import matplotlib.pyplot as plt
from SpectralNorm import *
import sys
import numpy as np

import os
import sys
import glob
import pickle
import matplotlib.pyplot as plt

def attention_block(input_tensor, input_channels=None, encoder_depth=1):
    if input_channels is None:
        input_channels = input_tensor.get_shape()[-1].value

    output_trunk = input_tensor

    output_soft_mask = MaxPooling2D(padding='same')(input_tensor)
    
    skip_connections = []
    for i in range(encoder_depth - 1):
        skip_connections.append(output_soft_mask)
        output_soft_mask = MaxPooling2D(padding='same')(output_soft_mask)

    skip_connections = list(reversed(skip_connections))

    for i in range(encoder_depth - 1):
        output_soft_mask = UpSampling2D()(output_soft_mask)
        output_soft_mask = Add()([output_soft_mask, skip_connections[i]])

    output_soft_mask = UpSampling2D()(output_soft_mask)

    # Output
    # Attention: (1 + output_soft_mask) * output_trunk
    output_soft_mask = Conv2D(input_channels, (1, 1), kernel_initializer='glorot_uniform')(output_soft_mask)
    output_soft_mask = Activation('sigmoid')(output_soft_mask)
    output = Lambda(lambda x: x + 1)(output_soft_mask)
    output = Multiply()([output, output_trunk])

    return output


def decode(layer_input, filters, f_size=4):

    u = UpSampling2D((2,2))(layer_input)
    u = ConvSN2D(filters, kernel_size=f_size, strides=1, kernel_initializer='glorot_uniform', padding='same')(u)
    u = BatchNormalization()(u)
    u = Activation('relu')(u)

    return u


def encode(layer_input, filters, f_size=4):

    d = ConvSN2D(filters, kernel_size=f_size, strides=2, kernel_initializer='glorot_uniform', padding='same')(layer_input)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)

    return d


def res_decode(layer_input, skip_input, filters, f_size=4):

    s = UpSampling2D((2,2))(skip_input)
    att = attention_block(s)
    u = Concatenate()([layer_input, att])
        
    u = UpSampling2D((2,2))(u)    
    u = ConvSN2D(filters, kernel_size=f_size, strides=1, kernel_initializer='glorot_uniform', padding='same')(u)
    u = BatchNormalization()(u)
    u = Activation('relu')(u)
    
    return u


def res_encode(skip_input, layer_input, filters, f_size=4):

    s = AveragePooling2D((2,2))(skip_input)
    att = attention_block(s)

    u = Concatenate()([layer_input, att])

    u = AveragePooling2D((2,2))(u)
    u = ConvSN2D(filters, kernel_size=f_size, strides=1, kernel_initializer='glorot_uniform', padding='same')(u)
    u = BatchNormalization()(u)
    u = LeakyReLU(alpha=0.2)(u)

    return u


def make_image(layer_input, f_size=4):
    img = ConvSN2D(3, kernel_size=f_size, strides=1, kernel_initializer='glorot_uniform', padding='same')(layer_input)
    img = Activation('tanh')(img)

    return img


def dis_layer_without(layer_input, filters, f_size=4):
    d = ConvSN2D(filters, kernel_size=f_size, strides=2, kernel_initializer='glorot_uniform', padding='same')(
        layer_input)
    d = LeakyReLU(alpha=0.2)(d)
    
    return d

def dis_layer(layer_input, filters, f_size=4):
    d = ConvSN2D(filters, kernel_size=f_size, strides=2, kernel_initializer='glorot_uniform', padding='same')(
        layer_input)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    
    return d


Using TensorFlow backend.


In [ ]:
class sentemb():
    def __init__(self):
        # Input shape
        self.img_rows = 128
        self.img_cols = 128
        self.channels = 3
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 128
        
        optimizer = Adam(0.00005, 0.5)
        
        self.discriminator = self.build_discriminator(32)
        self.discriminator.compile(loss='binary_crossentropy',
                                   optimizer=optimizer,
                                   metrics=['accuracy'])

        noise = Input(shape=(self.latent_dim,))
        sentence_emb = Input(shape=(512,))
        
        self.generator = self.build_generator(noise, sentence_emb, 64)

        img = self.generator([noise, sentence_emb])

        self.discriminator.trainable = False

        valid = self.discriminator([img, sentence_emb])
        
        optimizer = Adam(0.0002, 0.5)
        
        self.combined = Model([noise, sentence_emb], valid)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)
        self.combined.summary()
        try:
            self.combined.load_weights("result/trained_model_combined.h5")
            self.generator.load_weights("result/trained_model_generator.h5")
            self.discriminator.load_weights("result/trained_model_discrimiator.h5")
        except:
            print("wrong")
            pass
        
    def build_generator(self, noise, sentence_emb, channel):

        noise0 = Dense(1024)(noise)
        sentence_emb0 = Dense(1024)(sentence_emb)

        in_vector = Concatenate()([noise0, sentence_emb0])
        in_vector = Activation('relu')(in_vector)

        z0 = Dense(channel * 32 * 4 * 4, activation="relu")(in_vector)
        z1 = Reshape((4, 4, channel * 32))(z0)
        
        '''
        #en0 = encode(z1, channel * 64)
        de0 = decode(z1, channel * 32)
        res_de0 = res_decode(de0, z1, channel * 16)

        res_de1 = res_encode(res_de0, de0, channel * 16)
        de1 = decode(res_de1, channel * 8)
        res_en1 = res_decode(de1, res_de1, channel * 16)

        res_de2 = res_encode(res_en1, de1, channel * 8)
        de2 = decode(res_de2, channel * 4)
        res_en2 = res_decode(de2, res_de2, channel * 8)

        res_de3 = res_encode(res_en2, de2, channel * 4)
        de3 = decode(res_de3, channel * 2)
        res_en3 = res_decode(de3, res_de3, channel * 4)
        
        de4 = decode(res_en3, channel *2)
        output = make_image(de4)
        '''
        de0 = decode(z1, channel * 16)
        res_de0 = res_decode(de0, z1, channel * 8)
        res_de1 = res_decode(res_de0, de0, channel * 4)
        #res_de2 = res_decode(res_de1, res_de0, channel * 4)
        de1 = decode(res_de1, channel * 4)
        #res_de3 = res_decode(res_de2, res_de1, channel * 4)
        de2 = decode(de1, channel * 2)
        output = make_image(de2)
        
        out_model = Model([noise, sentence_emb], output)
        out_model.summary()

        return out_model

    def build_discriminator(self, channel):

        img = Input(shape=self.img_shape)
        #res_en0 = res_encode(img, res_de3, channel * 1)
        
        d0 = dis_layer_without(img, channel * 1)
        d1 = res_encode(img,  d0, channel * 2)
        d2 = res_encode(d0,  d1, channel * 4)
        d3 = res_encode(d1,  d2, channel * 8)
        d4 = res_encode(d2,  d3, channel * 16)
        d5 = res_encode(d3,  d4, channel * 32)
        d6 = dis_layer(d5, channel * 64)
        
        '''
        d1 = dis_layer(d0, channel * 2)
        d2 = dis_layer(d1, channel * 4)
        d3 = dis_layer(d2, channel * 8)
        d4 = dis_layer(d3, channel * 16)
        d5 = dis_layer(d4, channel * 32)
        d6 = dis_layer(d5, channel * 64)
        #d6 = AveragePooling2D((2,2))(d5)
        '''
        f = Flatten()(d6)
        f0 = DenseSN(1024,kernel_initializer='glorot_uniform')(f)
        f1 = LeakyReLU(alpha=0.2)(f0)

        text = Input(shape=(512,))
        text0 = DenseSN(1024,kernel_initializer='glorot_uniform')(text)
        text1 = LeakyReLU(alpha=0.2)(text0)

        output = Concatenate()([f1, text1])
        output = DenseSN(1,kernel_initializer='glorot_uniform')(output)
        output = Activation('sigmoid')(output)

        out_model = Model([img, text], output)
        out_model.summary()

        return out_model

    def train(self, epochs, batch_size=192, save_interval=50):

        caption_path = "102flowers/text_emb/"

        caption_list = []
        img_train = []
        
        
        for file_name in glob.glob(caption_path + "/*"):

            fileName, fileExtension = os.path.splitext(file_name)
            t_name = fileName.split('/')
            t_name = t_name[len(t_name)-1]
            print(fileName)
            with open(file_name, 'rb') as read_emb:
                for l in pickle.load(read_emb):
                    caption_list.append( l )
                    img_train.append( np.array(image.load_img("102flowers/jpg/" + t_name + ".jpg", target_size=(128,128))) / 127.5 - 1 )

        caption_list = np.array(caption_list)
        img_train = np.array(img_train)

        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))
        max_batch = int(caption_list.shape[0] / batch_size)
        
        from random import randint
        
        for epoch in range(epochs):
            #arr = [True for i in range(len(caption_list)-max_batch-1)]
            
            for i in range( max_batch ):
                
                #while(1):
                rdn = randint(0, (len(caption_list)-batch_size-1))
                    #if arr[rdn]:
                        #arr[rdn] = False
                        #break
                        
                batch_x = caption_list[rdn : rdn + batch_size]
                batch_y = img_train[rdn : rdn + batch_size]

                noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
                gen_imgs = self.generator.predict([noise, batch_x])
                
                self.discriminator.trainable = True
                d_loss_real = self.discriminator.train_on_batch( [batch_y, batch_x], valid)
                d_loss_fake = self.discriminator.train_on_batch( [gen_imgs, batch_x], fake)

                d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
                self.discriminator.trainable = False
                
                g_loss = self.combined.train_on_batch( [noise, batch_x], valid)
                print("%d %d / %d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, i, max_batch, d_loss[0], 100 * d_loss[1], g_loss))
            
            self.combined.save_weights("result/trained_model_combined.h5")
            self.generator.save_weights("result/trained_model_generator.h5")
            self.discriminator.save_weights("result/trained_model_discrimiator.h5")
                
            if epoch % save_interval == 0:
                
                r, c = 24, 8
                noise = np.random.normal(0, 1, (r * c, self.latent_dim))

                gen_imgs = self.generator.predict([noise, batch_x])

                # Rescale images 0 - 1
                gen_imgs = 0.5 * gen_imgs + 0.5
                
                fig, axs = plt.subplots(r, c)
                cnt = 0
                for i in range(r):
                    for j in range(c):
                        image.save_img("result/%d_%d.jpg"%(epoch, cnt), gen_imgs[cnt])
                        axs[i, j].imshow(gen_imgs[cnt])
                        axs[i, j].axis('off')
                        cnt = cnt + 1
                        
                fig.savefig("result/flower_%d.png" % epoch)
                plt.close()




In [ ]:
if __name__ == '__main__':
    
    sent = sentemb()
    sent.train(500000)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 128, 128, 3)  0                                            
__________________________________________________________________________________________________
average_pooling2d_1 (AveragePoo (None, 64, 64, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 32, 32, 3)    0           average_pooling2d_1[0][0]        
__________________________________________________________________________________________________
up_sampling2d_1 (UpSampling2D)  (None, 64, 64, 3)    0           max_pooling2d_1[0][0]            
__________________________________________________________________________________________________
conv2d_1 (

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 128)          0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, 512)          0                                            
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 1024)         132096      input_3[0][0]                    
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 1024)         525312      input_4[0][0]                    
__________________________________________________________________________________________________
concatenat

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 128)          0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, 512)          0                                            
__________________________________________________________________________________________________
model_2 (Model)                 (None, 128, 128, 3)  139648774   input_3[0][0]                    
                                                                 input_4[0][0]                    
__________________________________________________________________________________________________
model_1 (Model)                 (None, 1)            53042862    model_2[1][0]                    
          

102flowers/text_emb/image_03417
102flowers/text_emb/image_01891
102flowers/text_emb/image_03242
102flowers/text_emb/image_02502
102flowers/text_emb/image_03370
102flowers/text_emb/image_03380
102flowers/text_emb/image_06900
102flowers/text_emb/image_01942
102flowers/text_emb/image_01934
102flowers/text_emb/image_02192
102flowers/text_emb/image_03265
102flowers/text_emb/image_02247
102flowers/text_emb/image_07010
102flowers/text_emb/image_01633
102flowers/text_emb/image_01629
102flowers/text_emb/image_03444
102flowers/text_emb/image_03401
102flowers/text_emb/image_06923
102flowers/text_emb/image_05905
102flowers/text_emb/image_03208
102flowers/text_emb/image_02476
102flowers/text_emb/image_02267
102flowers/text_emb/image_01883
102flowers/text_emb/image_03904
102flowers/text_emb/image_02249
102flowers/text_emb/image_01632
102flowers/text_emb/image_01909
102flowers/text_emb/image_02497
102flowers/text_emb/image_05945
102flowers/text_emb/image_03431
102flowers/text_emb/image_03879
102flowe

102flowers/text_emb/image_01619
102flowers/text_emb/image_02533
102flowers/text_emb/image_01880
102flowers/text_emb/image_02205
102flowers/text_emb/image_02315
102flowers/text_emb/image_06918
102flowers/text_emb/image_01599
102flowers/text_emb/image_01914
102flowers/text_emb/image_04719
102flowers/text_emb/image_02541
102flowers/text_emb/image_03874
102flowers/text_emb/image_01832
102flowers/text_emb/image_06895
102flowers/text_emb/image_01608
102flowers/text_emb/image_04753
102flowers/text_emb/image_03235
102flowers/text_emb/image_08065
102flowers/text_emb/image_04744
102flowers/text_emb/image_03185
102flowers/text_emb/image_01916
102flowers/text_emb/image_04752
102flowers/text_emb/image_07018
102flowers/text_emb/image_03440
102flowers/text_emb/image_03272
102flowers/text_emb/image_03897
102flowers/text_emb/image_01647
102flowers/text_emb/image_03870
102flowers/text_emb/image_01851
102flowers/text_emb/image_02242
102flowers/text_emb/image_01607
102flowers/text_emb/image_03311
102flowe

...

136 15 / 58 [D loss: 0.016139, acc.: 100.00%] [G loss: 0.020218]
136 16 / 58 [D loss: 0.030639, acc.: 100.00%] [G loss: 0.012780]
136 17 / 58 [D loss: 0.011928, acc.: 100.00%] [G loss: 0.015411]
136 18 / 58 [D loss: 0.011281, acc.: 100.00%] [G loss: 0.015578]
136 19 / 58 [D loss: 0.031590, acc.: 100.00%] [G loss: 0.122196]
136 20 / 58 [D loss: 0.027537, acc.: 100.00%] [G loss: 0.182987]
136 21 / 58 [D loss: 0.029489, acc.: 100.00%] [G loss: 0.069986]
136 22 / 58 [D loss: 0.023052, acc.: 100.00%] [G loss: 0.037125]
136 23 / 58 [D loss: 0.020567, acc.: 100.00%] [G loss: 0.029004]
136 24 / 58 [D loss: 0.025003, acc.: 100.00%] [G loss: 0.039347]
136 25 / 58 [D loss: 0.020283, acc.: 100.00%] [G loss: 0.025914]
136 26 / 58 [D loss: 0.014247, acc.: 100.00%] [G loss: 0.025814]
136 27 / 58 [D loss: 0.019584, acc.: 100.00%] [G loss: 0.022077]
136 28 / 58 [D loss: 0.013615, acc.: 100.00%] [G loss: 0.022661]
136 29 / 58 [D loss: 0.008718, acc.: 100.00%] [G loss: 0.021449]
136 30 / 58 [D loss: 0.02

138 26 / 58 [D loss: 0.012785, acc.: 100.00%] [G loss: 0.004164]
138 27 / 58 [D loss: 0.008811, acc.: 100.00%] [G loss: 0.005266]
138 28 / 58 [D loss: 0.008060, acc.: 100.00%] [G loss: 0.005615]
138 29 / 58 [D loss: 0.007759, acc.: 100.00%] [G loss: 0.005353]
138 30 / 58 [D loss: 0.009157, acc.: 100.00%] [G loss: 0.004638]
138 31 / 58 [D loss: 0.011321, acc.: 100.00%] [G loss: 0.007335]
138 32 / 58 [D loss: 0.009185, acc.: 100.00%] [G loss: 0.006943]
138 33 / 58 [D loss: 0.009012, acc.: 100.00%] [G loss: 0.005080]
138 34 / 58 [D loss: 0.007870, acc.: 100.00%] [G loss: 0.006085]
138 35 / 58 [D loss: 0.012201, acc.: 100.00%] [G loss: 0.005982]
138 36 / 58 [D loss: 0.010325, acc.: 100.00%] [G loss: 0.006092]
138 37 / 58 [D loss: 0.009713, acc.: 100.00%] [G loss: 0.005862]
138 38 / 58 [D loss: 0.011795, acc.: 100.00%] [G loss: 0.006532]
138 39 / 58 [D loss: 0.008741, acc.: 100.00%] [G loss: 0.007081]
138 40 / 58 [D loss: 0.008125, acc.: 100.00%] [G loss: 0.006826]
138 41 / 58 [D loss: 0.01

140 37 / 58 [D loss: 0.019171, acc.: 100.00%] [G loss: 0.003605]
140 38 / 58 [D loss: 0.010295, acc.: 100.00%] [G loss: 0.004228]
140 39 / 58 [D loss: 0.011244, acc.: 100.00%] [G loss: 0.003635]
140 40 / 58 [D loss: 0.014834, acc.: 100.00%] [G loss: 0.003037]
140 41 / 58 [D loss: 0.007897, acc.: 100.00%] [G loss: 0.002950]
140 42 / 58 [D loss: 0.007696, acc.: 100.00%] [G loss: 0.003396]
140 43 / 58 [D loss: 0.005387, acc.: 100.00%] [G loss: 0.003249]
140 44 / 58 [D loss: 0.005850, acc.: 100.00%] [G loss: 0.003359]
140 45 / 58 [D loss: 0.018033, acc.: 100.00%] [G loss: 0.003659]
140 46 / 58 [D loss: 0.007251, acc.: 100.00%] [G loss: 0.003391]
140 47 / 58 [D loss: 0.006880, acc.: 100.00%] [G loss: 0.003307]
140 48 / 58 [D loss: 0.009601, acc.: 100.00%] [G loss: 0.003688]
140 49 / 58 [D loss: 0.008149, acc.: 100.00%] [G loss: 0.002688]
140 50 / 58 [D loss: 0.007772, acc.: 100.00%] [G loss: 0.003576]
140 51 / 58 [D loss: 0.008240, acc.: 100.00%] [G loss: 0.002872]
140 52 / 58 [D loss: 0.01

142 48 / 58 [D loss: 0.004108, acc.: 100.00%] [G loss: 0.005558]
142 49 / 58 [D loss: 0.004565, acc.: 100.00%] [G loss: 0.006284]
142 50 / 58 [D loss: 0.004564, acc.: 100.00%] [G loss: 0.005376]
142 51 / 58 [D loss: 0.004076, acc.: 100.00%] [G loss: 0.005802]
142 52 / 58 [D loss: 0.005702, acc.: 100.00%] [G loss: 0.005252]
142 53 / 58 [D loss: 0.005294, acc.: 100.00%] [G loss: 0.006583]
142 54 / 58 [D loss: 0.009164, acc.: 100.00%] [G loss: 0.004514]
142 55 / 58 [D loss: 0.006609, acc.: 100.00%] [G loss: 0.004525]
142 56 / 58 [D loss: 0.004889, acc.: 100.00%] [G loss: 0.005869]
142 57 / 58 [D loss: 0.005234, acc.: 100.00%] [G loss: 0.006085]
143 0 / 58 [D loss: 0.004398, acc.: 100.00%] [G loss: 0.005172]
143 1 / 58 [D loss: 0.006230, acc.: 100.00%] [G loss: 0.005536]
143 2 / 58 [D loss: 0.004886, acc.: 100.00%] [G loss: 0.005781]
143 3 / 58 [D loss: 0.008423, acc.: 100.00%] [G loss: 0.005078]
143 4 / 58 [D loss: 0.004948, acc.: 100.00%] [G loss: 0.005575]
143 5 / 58 [D loss: 0.004533, 

145 1 / 58 [D loss: 0.006850, acc.: 100.00%] [G loss: 0.015503]
145 2 / 58 [D loss: 0.007123, acc.: 100.00%] [G loss: 0.018249]
145 3 / 58 [D loss: 0.006935, acc.: 100.00%] [G loss: 0.017219]
145 4 / 58 [D loss: 0.007913, acc.: 100.00%] [G loss: 0.015241]
145 5 / 58 [D loss: 0.008546, acc.: 100.00%] [G loss: 0.014099]
145 6 / 58 [D loss: 0.006151, acc.: 100.00%] [G loss: 0.014579]
145 7 / 58 [D loss: 0.007126, acc.: 100.00%] [G loss: 0.016540]
145 8 / 58 [D loss: 0.005853, acc.: 100.00%] [G loss: 0.014640]
145 9 / 58 [D loss: 0.009332, acc.: 100.00%] [G loss: 0.017862]
145 10 / 58 [D loss: 0.007518, acc.: 100.00%] [G loss: 0.018715]
145 11 / 58 [D loss: 0.008268, acc.: 100.00%] [G loss: 0.018489]
145 12 / 58 [D loss: 0.005959, acc.: 100.00%] [G loss: 0.021506]
145 13 / 58 [D loss: 0.005918, acc.: 100.00%] [G loss: 0.017920]
145 14 / 58 [D loss: 0.006696, acc.: 100.00%] [G loss: 0.017942]
145 15 / 58 [D loss: 0.006615, acc.: 100.00%] [G loss: 0.016246]
145 16 / 58 [D loss: 0.006921, acc